In [1]:
import sys
sys.path.append('..')
import numpy as np
from harang import utils, vision
from tensorflow.keras.models import load_model
import efficientnet.tfkeras as eff
import os
import gc

<IPython.core.display.Javascript object>

W1114 17:41:46.577456 20588 deprecation_wrapper.py:119] From ..\harang\vision.py:22: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1114 17:41:46.579457 20588 deprecation_wrapper.py:119] From ..\harang\vision.py:24: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1114 17:41:47.398633 20588 deprecation_wrapper.py:119] From ..\harang\vision.py:25: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.



In [2]:
pseudo = 'stage1'

# Classification

In [3]:
wd = f'{pseudo}/models/cls'
paths = [f'{wd}/{p}' for p in os.listdir(wd)]

In [3]:
test = utils.from_pickle('data/unlabeled.pkl')
test_pred = np.zeros((len(test),4), dtype=np.float32)
for path in paths:
    print(path)
    model = load_model(path, compile=False)
    tmp_test_pred = np.zeros((len(test),4), dtype=np.float32)
    for hf in [{'p': 1.0}, False]:
        for vf in [{'p': 1.0}, False]:
            tta_generator = vision.Generator(
                test,
                batch_size=64,
                augment='image',
                hf=hf,
                vf=vf,
                preprocess_input=eff.preprocess_input
            )
            tmp_test_pred += model.predict_generator(tta_generator, verbose=1)/4
    test_pred += tmp_test_pred/len(paths)

W1113 21:26:02.470226 19140 deprecation.py:506] From c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1113 21:26:02.471226 19140 deprecation.py:573] From c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\util\deprecation.py:507: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with distribution=normal is deprecated and will be removed in a future version.
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
W1113 21:26:02.471226 19140 deprecation.py:506] From c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future versio

449/449 [==============================] - 173s 385ms/step


In [4]:
utils.to_pickle(f'{pseudo}/unlabeled_pred/cls/pred.pkl', test_pred)

# Segmentation

In [3]:
wd = f'{pseudo}/models/seg'
paths = [f'{wd}/{p}' for p in os.listdir(wd)]
paths.reverse()

In [ ]:
test = utils.from_pickle('data/unlabeled.pkl')
test_pred = np.zeros((len(test), 320, 480, 4), dtype=np.float16)
n_chunks = 8
for path in paths:
    print(path)
    model = load_model(path, compile=False)
    for chunk_indices in np.array_split(range(len(test)), n_chunks):
        for hf in [{'p': 1.0}, False]:
            for vf in [{'p': 1.0}, False]:
                tta_generator = vision.Generator(
                    test[chunk_indices],
                    batch_size=32,
                    augment='image',
                    hf=hf,
                    vf=vf,
                    preprocess_input=eff.preprocess_input
                )
                single_pred = model.predict_generator(tta_generator, verbose=1)[0]
                if hf:
                    single_pred = np.flip(single_pred, axis=2)
                if vf:
                    single_pred = np.flip(single_pred, axis=1)
                test_pred[chunk_indices] += single_pred/(4*len(paths))
                del single_pred; gc.collect();

W1114 17:42:04.153389 20588 deprecation.py:506] From c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1114 17:42:04.153389 20588 deprecation.py:573] From c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\util\deprecation.py:507: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with distribution=normal is deprecated and will be removed in a future version.
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
W1114 17:42:04.154390 20588 deprecation.py:506] From c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future versio

stage1/models/seg/efficientnetb4_4_0.6375.h5


W1114 17:42:04.416449 20588 deprecation.py:506] From c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


113/113 [==============================] - 38s 332ms/step
stage1/models/seg/efficientnetb4_3_0.6377.h5
113/113 [==============================] - 38s 333ms/step
stage1/models/seg/efficientnetb4_2_0.6452.h5
113/113 [==============================] - 38s 337ms/step
stage1/models/seg/efficientnetb4_1_0.6384.h5
113/113 [==============================] - 38s 334ms/step
stage1/models/seg/efficientnetb4_0_0.6418.h5
113/113 [==============================] - 39s 349ms/step
stage1/models/seg/efficientnetb3_4_0.6394.h5
113/113 [==============================] - 34s 300ms/step
stage1/models/seg/efficientnetb3_3_0.6440.h5
113/113 [==============================] - 34s 299ms/step
stage1/models/seg/efficientnetb3_2_0.6487.h5
113/113 [==============================] - 33s 291ms/step
stage1/models/seg/efficientnetb3_1_0.6437.h5
 61/113 [===============>..............] - ETA: 13s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



stage1/models/seg/efficientnetb3_0_0.6456.h5
113/113 [==============================] - 33s 292ms/step
stage1/models/seg/efficientnetb2_4_0.6353.h5
113/113 [==============================] - 29s 258ms/step


In [ ]:
utils.to_pickle(f'{pseudo}/unlabeled_pred/seg/pred.pkl', test_pred)